In [2]:
include("/Users/meesvandartel/Desktop/Coursework/CGT/DeepEWA/FastEWA.jl")
using Flux, .fEWA, Random, IterTools, ProgressMeter

In [7]:
# this data will be for this specific coordination game with 2 NE and 1 mixed NE.
coord = [[[5 1; 1 4], [5 1; 1 4]], fEWA.find_NE_mixed([[5 1; 1 4], [5 1; 1 4]])]
points = 1000000 # N of observations to generate

α_grid = rand(0.0:0.0001:1.0, points) 
κ_grid = rand(0.0:0.0001:1.0, points)
δ_grid = rand(0.0:0.0001:1.0, points)
β_grid = exp.(rand(0.0:0.001:4.0, points)) # take exp to get the lim to infty effect since β is unbounded above
combs = [[α_grid[i], κ_grid[i], δ_grid[i], β_grid[i]] for i in 1:points]
subset=Int64(floor(0.6*length(combs))) #60/40 train test split, since we have many obs

x_train, x_test = combs[1:subset], combs[(subset+1):end]
y_train, y_test = Vector{Tuple{Array{Int}, Array{Array{Float64}}, Array{Array{Float64}}, Bool}}(undef, length(x_train)), Vector{Tuple{Array{Int}, Array{Array{Float64}}, Array{Array{Float64}}, Bool}}(undef, length(x_test))
println("Finished")

Finished


In [6]:
# should take about 6min for 1mil obs
@showprogress for i in 1:length(x_train)
    comb=x_train[i]
    α, κ, δ, β = comb
    params = fEWA.init_EWA(;α=α, κ=κ, δ=δ, β=β,game=coord)
    sₜ, σ, Qₜ, NE_found  = fEWA.Run_FastEWA(params)
    y_train[i] = (sₜ, σ, Qₜ, NE_found)
end

@showprogress for i in 1:length(x_test)
    comb = x_test[i]
    α, κ, δ, β = comb
    params = fEWA.init_EWA(;α=α, κ=κ, δ=δ, β=β,game=coord)
    sₜ, σ, Qₜ, NE_found  = fEWA.Run_FastEWA(params)
    y_test[i] = (sₜ, σ, Qₜ, NE_found)
end

Progress: 100%|█████████████████████████████████████████| Time: 0:02:599:26
Progress: 100%|█████████████████████████████████████████| Time: 0:02:02
